In [2]:
# Imports
import os
import json
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Layout, Dropdown, BoundedIntText, BoundedFloatText,
    FloatText, Checkbox, Button, Text, Label, Tab, HTML
)
from IPython.display import display


## Configurations & Default values


In [3]:
# Configuration
SAVE_FILE = 'simulation_setup.json'

# Default values
DEFAULTS = {
    "Metadata": {
        "num_processors": 4,
        "debug_output_frequency": 10
    },
    "PottsCore": {
        "dim_x": 100,
        "dim_y": 100,
        "dim_z": 1,
        "steps": 100000,
        "anneal": 0,
        "fluctuation_amplitude": 10.0,
        "fluctuation_amplitude_function": "Min",
        "boundary_x": "NoFlux",
        "boundary_y": "NoFlux",
        "boundary_z": "NoFlux",
        "neighbor_order": 1,
        "random_seed": None,
        "lattice_type": "Cartesian",
        "offset": 0
    },
    "CellType": [
        {"Cell type": "Medium", "freeze": False}
    ],
    "Constraints": {
        "Volume": [
            {"CellType": "Medium", "enabled": False, "target_volume": 25.0, "lambda_volume": 2.0}
        ],
        "Surface": []
    }
}

## Helper function
Functions that help save, load, and update parameter values in all UIs.

In [4]:
# Cell 3: Helper Functions
def load_saved_values():
    """Load saved values or use defaults"""
    try:
        if os.path.exists(SAVE_FILE):
            with open(SAVE_FILE, 'r') as f:
                return json.load(f)
        return DEFAULTS.copy()
    except (json.JSONDecodeError, IOError):
        print("⚠️ JSON file is corrupted or inaccessible. Resetting to defaults.")
        return DEFAULTS.copy()

def current_config(widgets_dict, celltype_entries, constraints):
    """Get current configuration from widgets"""
    return {
        "Metadata": {
            "num_processors": widgets_dict["num_proc"].value,
            "debug_output_frequency": widgets_dict["debug_freq"].value
        },
        "PottsCore": {
            "dim_x": widgets_dict["x_slider"].value,
            "dim_y": widgets_dict["y_slider"].value,
            "dim_z": widgets_dict["z_slider"].value,
            "steps": widgets_dict["steps_input"].value,
            "anneal": widgets_dict["anneal_input"].value,
            "fluctuation_amplitude": widgets_dict["fluctuation_slider"].value,
            "fluctuation_amplitude_function": widgets_dict["flunct_fn_dropdown"].value,
            "boundary_x": widgets_dict["boundary_x"].value,
            "boundary_y": widgets_dict["boundary_y"].value,
            "boundary_z": widgets_dict["boundary_z"].value,
            "neighbor_order": widgets_dict["neighbor_order_input"].value,
            "random_seed": int(widgets_dict["seed_input"].value) if widgets_dict["seed_input"].value.strip() else None,
            "lattice_type": 'Cartesian' if widgets_dict["lattice_dropdown"].value == 'Square' else 'Hexagonal',
            "offset": widgets_dict["offset_input"].value
        },
        "CellType": celltype_entries,
        "Constraints": constraints
    }

def save_to_json(config):
    """Save current configuration to JSON"""
    with open(SAVE_FILE, 'w') as f:
        json.dump(config, f, indent=4)

def update_celltype_display(celltype_entries, display_box):
    """Update the cell type display box"""
    items = []
    for i, entry in enumerate(celltype_entries):
        label_str = entry["Cell type"]
        if entry.get("freeze", False):
            label_str += " (frozen)"
        label = Label(label_str, layout=Layout(flex='1 1 auto'))
        remove_btn = Button(description="Remove", button_style='danger', layout=Layout(width='80px'))
        
        def make_remove_handler(index, celltype_entries, display_box):
            def handler(_):
                del celltype_entries[index]
                update_celltype_display(celltype_entries, display_box)
                save_to_json(current_config(widgets_dict, celltype_entries, constraints))
            return handler
        
        remove_btn.on_click(make_remove_handler(i, celltype_entries, display_box))
        items.append(HBox([label, remove_btn], layout=Layout(justify_content='space-between')))
    
    display_box.children = items

## UI
This is where all the Widget are created. 
- Metadata
- Potts
- Cell Type
- Volume & Surface Constraint

In [5]:
# Cell 4: UI Creation Functions
# Initialize state
saved_values = load_saved_values()
celltype_entries = saved_values.get("CellType", DEFAULTS["CellType"].copy())
constraints = saved_values.get("Constraints", DEFAULTS["Constraints"].copy())

# Create widget dictionaries
widgets_dict = {}

# Metadata widgets
widgets_dict["num_proc"] = widgets.IntText(
    value=saved_values["Metadata"]["num_processors"], min=1,
    description='Number of Processors:',
    style={'description_width': 'initial'}
)
widgets_dict["debug_freq"] = widgets.IntText(
    value=saved_values["Metadata"]["debug_output_frequency"], min=1,
    description='Debug Output Frequency:',
    style={'description_width': 'initial'}
)

# PottsCore widgets
widgets_dict["x_slider"] = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_x"],
    min=0, max=100, description='X:',
    continuous_update=False,
    style={'description_width': 'initial'}
)
widgets_dict["y_slider"] = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_y"],
    min=0, max=100, description='Y:',
    continuous_update=False,
    style={'description_width': 'initial'}
)
widgets_dict["z_slider"] = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_z"],
    min=0, max=100, description='Z:',
    continuous_update=False,
    style={'description_width': 'initial'}
)
widgets_dict["steps_input"] = widgets.IntText(
    value=saved_values["PottsCore"]["steps"],
    description='MC Steps:',
    style={'description_width': 'initial'}
)
widgets_dict["anneal_input"] = widgets.FloatText(
    value=saved_values["PottsCore"]["anneal"],
    description='Anneal:',
    style={'description_width': 'initial'}
)
widgets_dict["fluctuation_slider"] = widgets.FloatSlider(
    value=saved_values["PottsCore"]["fluctuation_amplitude"],
    min=0.0, max=50.0, step=0.1,
    description='Fluctuation:',
    continuous_update=False,
    style={'description_width': 'initial'}
)
widgets_dict["flunct_fn_dropdown"] = widgets.Dropdown(
    options=['Min', 'Max', 'ArithmeticAverage'],
    value=saved_values["PottsCore"]["fluctuation_amplitude_function"],
    description='Fluctuation Function:',
    style={'description_width': 'initial'}
)
widgets_dict["boundary_x"] = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_x"],
    description='Boundary X:',
    style={'description_width': 'initial'}
)
widgets_dict["boundary_y"] = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_y"],
    description='Boundary Y:',
    style={'description_width': 'initial'}
)
widgets_dict["boundary_z"] = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_z"],
    description='Boundary Z:',
    style={'description_width': 'initial'}
)
widgets_dict["neighbor_order_input"] = widgets.BoundedIntText(
    value=saved_values["PottsCore"]["neighbor_order"],
    min=1, max=20, description='Neighbor Order:',
    style={'description_width': 'initial'}
)
widgets_dict["seed_input"] = widgets.Text(
    value='' if saved_values["PottsCore"]["random_seed"] is None 
         else str(saved_values["PottsCore"]["random_seed"]),
    description='Random Seed:',
    placeholder='e.g. 123456',
    style={'description_width': 'initial'}
)
widgets_dict["lattice_dropdown"] = widgets.Dropdown(
    options=['Square', 'Hexagonal'],
    value='Square' if saved_values["PottsCore"]["lattice_type"] == 'Cartesian' 
          else 'Hexagonal',
    description='Lattice Type:',
    style={'description_width': 'initial'}
)
widgets_dict["offset_input"] = widgets.IntText(
    value=saved_values["PottsCore"]["offset"],
    description='Offset:',
    style={'description_width': 'initial'}
)

# CellType widgets
widgets_dict["celltype_display_box"] = VBox(
    layout=Layout(border='1px solid gray', padding='10px', width='300px')
)
update_celltype_display(celltype_entries, widgets_dict["celltype_display_box"])

widgets_dict["preset_dropdown"] = Dropdown(
    options=["Medium", "Condensing", "NonCondensing", "Customize"],
    value="Medium",
    description="Cell Type:",
    style={'description_width': 'initial'},
    layout=Layout(width='250px')
)
widgets_dict["freeze_checkbox"] = Checkbox(
    value=True,
    description="Freeze",
    indent=False,
    layout=Layout(margin='0 0 0 20px')
)
widgets_dict["custom_name_input"] = Text(
    description="Name:",
    placeholder="e.g. T1",
    style={'description_width': 'initial'},
    layout=Layout(width='250px')
)
widgets_dict["custom_name_input"].layout.display = 'none'
widgets_dict["add_button"] = Button(
    description="Add",
    button_style="success",
    layout=Layout(width='80px', margin='10px 0 0 0')
)

# Constraints widgets
widgets_dict["constraints_display_box"] = VBox(
    layout=Layout(border='1px solid gray', padding='10px', width='500px'))
widgets_dict["constraints_celltype_dropdown"] = widgets.Dropdown(
    options=[entry["Cell type"] for entry in celltype_entries],
    description='Cell Type:',
    style={'description_width': 'initial'}
)
widgets_dict["vol_enabled"] = widgets.Checkbox(
    value=False,
    description="Enable Volume Constraints",
    indent=False,
    style={'description_width': 'initial'}
)
widgets_dict["target_volume"] = widgets.FloatText(
    value=25.0,
    min=1.0,
    description='Target Volume:',
    style={'description_width': 'initial'}
)
widgets_dict["lambda_volume"] = widgets.FloatText(
    value=2.0,
    min=0.0,
    description='Lambda Volume:',
    style={'description_width': 'initial'}
)
widgets_dict["surf_enabled"] = widgets.Checkbox(
    value=False,
    description="Enable Surface Constraints",
    indent=False,
    style={'description_width': 'initial'}
)
widgets_dict["target_surface"] = widgets.FloatText(
    value=100.0,
    min=1.0,
    description='Target Surface:',
    style={'description_width': 'initial'}
)
widgets_dict["lambda_surface"] = widgets.FloatText(
    value=0.5,
    min=0.0,
    description='Lambda Surface:',
    style={'description_width': 'initial'}
)
widgets_dict["add_constraints_button"] = Button(
    description="Add Constraints",
    button_style="success",
    layout=Layout(width='150px')
)

# Reset buttons
widgets_dict["reset_metadata_button"] = Button(
    description="Reset Tab",
    button_style='warning',
    layout=Layout(width='100px')
)
widgets_dict["reset_potts_button"] = Button(
    description="Reset Tab",
    button_style='warning',
    layout=Layout(width='100px')
)
widgets_dict["reset_ct_button"] = Button(
    description="Reset Tab",
    button_style='warning',
    layout=Layout(width='100px')
)
widgets_dict["reset_constraints_button"] = Button(
    description="Reset Tab",
    button_style='warning',
    layout=Layout(width='100px')
)
widgets_dict["reset_button"] = Button(
    description="Reset All to Defaults",
    button_style='danger'
)

## Layout
Mainly how each wigets are formatted in the tabs

In [6]:
# Cell 5: UI Layout Functions
def create_metadata_tab():
    return VBox([
        widgets_dict["num_proc"],
        widgets_dict["debug_freq"],
        widgets_dict["reset_metadata_button"]
    ], layout=Layout(padding='10px'))

def create_potts_tab():
    return VBox([
        HTML("<b>Dimensions:</b>"),
        HBox([widgets_dict["x_slider"], widgets_dict["y_slider"], widgets_dict["z_slider"]]),
        HTML("<b>Core Parameters:</b>"),
        widgets_dict["steps_input"],
        widgets_dict["anneal_input"],
        widgets_dict["fluctuation_slider"],
        widgets_dict["flunct_fn_dropdown"],
        HTML("<b>Boundaries:</b>"),
        HBox([widgets_dict["boundary_x"], widgets_dict["boundary_y"], widgets_dict["boundary_z"]]),
        HTML("<b>Advanced:</b>"),
        widgets_dict["neighbor_order_input"],
        widgets_dict["lattice_dropdown"],
        widgets_dict["offset_input"],
        widgets_dict["seed_input"],
        widgets_dict["reset_potts_button"]
    ], layout=Layout(padding='10px'))

def create_celltype_tab():
    cell_type_row = HBox([
        widgets_dict["preset_dropdown"],
        widgets_dict["freeze_checkbox"]
    ])
    
    return VBox([
        HBox([
            VBox([
                Label("Current Cell Types:", style={'font_weight': 'bold'}),
                widgets_dict["celltype_display_box"],
                widgets_dict["reset_ct_button"]
            ], layout=Layout(width='320px', padding='0 20px 0 0')),
            VBox([
                Label("Add Cell Type:", style={'font_weight': 'bold'}),
                cell_type_row,
                widgets_dict["custom_name_input"],
                widgets_dict["add_button"]
            ])
        ])
    ], layout=Layout(padding='10px'))

def create_constraints_tab():
    return VBox([
        HBox([
            widgets_dict["constraints_celltype_dropdown"],
            widgets_dict["add_constraints_button"]
        ]),
        HTML("<hr><b>Volume Constraints:</b>"),
        widgets_dict["vol_enabled"],
        HBox([widgets_dict["target_volume"], widgets_dict["lambda_volume"]]),
        HTML("<hr><b>Surface Constraints:</b>"),
        widgets_dict["surf_enabled"],
        HBox([widgets_dict["target_surface"], widgets_dict["lambda_surface"]]),
        HTML("<hr><b>Current Constraints:</b>"),
        widgets_dict["constraints_display_box"],
        widgets_dict["reset_constraints_button"]
    ], layout=Layout(padding='10px'))

def setup_tabs():
    tabs = Tab()
    tabs.children = [
        create_metadata_tab(),
        create_potts_tab(),
        create_celltype_tab(),
        create_constraints_tab()
    ]
    tabs.set_title(0, 'Basic Setup')
    tabs.set_title(1, 'Potts Core')
    tabs.set_title(2, 'Cell Types')
    tabs.set_title(3, 'Constraints')
    return tabs

## Response to user 
Functions are defined to update information when items are add/removed or reset. Mostly functional buttons

In [7]:
# Cell 6: Event Handlers
def toggle_custom_input(change):
    widgets_dict["custom_name_input"].layout.display = 'block' if change['new'] == "Customize" else 'none'

def on_add_clicked(_):
    selected = widgets_dict["preset_dropdown"].value
    name = (widgets_dict["custom_name_input"].value.strip() 
            if selected == "Customize" else selected)
    
    if not name:
        widgets_dict["custom_name_input"].placeholder = "Please enter a name!"
        return
    
    celltype_entries.append({
        "Cell type": name, 
        "freeze": widgets_dict["freeze_checkbox"].value
    })
    
    update_celltype_display(celltype_entries, widgets_dict["celltype_display_box"])
    widgets_dict["constraints_celltype_dropdown"].options = [entry["Cell type"] for entry in celltype_entries]
    widgets_dict["custom_name_input"].value = ""
    save_to_json(current_config(widgets_dict, celltype_entries, constraints))

def update_constraints_display():
    """Update the constraints display box"""
    items = []
    
    # Volume constraints
    vol_header = Label("Volume Constraints:", style={'font_weight': 'bold', 'font_style': 'italic'})
    items.append(vol_header)
    
    for entry in constraints["Volume"]:
        ct = entry["CellType"]
        enabled = "✓" if entry["enabled"] else "✗"
        text = f"{ct}: Enabled={enabled}, Target={entry['target_volume']}, λ={entry['lambda_volume']:.2f}"
        label = Label(text, layout=Layout(flex='1 1 auto'))
        
        remove_btn = Button(description="Remove", button_style='danger', layout=Layout(width='80px'))
        
        def make_remove_handler(ct):
            def handler(_):
                constraints["Volume"] = [e for e in constraints["Volume"] if e["CellType"] != ct]
                constraints["Surface"] = [e for e in constraints["Surface"] if e["CellType"] != ct]
                save_to_json(current_config(widgets_dict, celltype_entries, constraints))
                update_constraints_display()
            return handler
        
        remove_btn.on_click(make_remove_handler(ct))
        items.append(HBox([label, remove_btn], layout=Layout(justify_content='space-between')))
    
    # Surface constraints
    surf_header = Label("Surface Constraints:", style={'font_weight': 'bold', 'font_style': 'italic'})
    items.append(surf_header)
    
    for entry in constraints["Surface"]:
        ct = entry["CellType"]
        enabled = "✓" if entry["enabled"] else "✗"
        text = f"{ct}: Enabled={enabled}, Target={entry['target_surface']}, λ={entry['lambda_surface']:.2f}"
        label = Label(text, layout=Layout(flex='1 1 auto'))
        
        remove_btn = Button(description="Remove", button_style='danger', layout=Layout(width='80px'))
        
        def make_remove_handler(ct):
            def handler(_):
                constraints["Surface"] = [e for e in constraints["Surface"] if e["CellType"] != ct]
                save_to_json(current_config(widgets_dict, celltype_entries, constraints))
                update_constraints_display()
            return handler
        
        remove_btn.on_click(make_remove_handler(ct))
        items.append(HBox([label, remove_btn], layout=Layout(justify_content='space-between')))
    
    if not constraints["Volume"] and not constraints["Surface"]:
        items.append(Label("No constraints defined"))
    
    widgets_dict["constraints_display_box"].children = items

def on_add_constraints_clicked(_):
    cell_type = widgets_dict["constraints_celltype_dropdown"].value
    
    # Add/update volume constraints
    vol_exists = False
    for entry in constraints["Volume"]:
        if entry["CellType"] == cell_type:
            entry.update({
                "enabled": widgets_dict["vol_enabled"].value,
                "target_volume": widgets_dict["target_volume"].value,
                "lambda_volume": widgets_dict["lambda_volume"].value
            })
            vol_exists = True
            break
    
    if not vol_exists:
        constraints["Volume"].append({
            "CellType": cell_type,
            "enabled": widgets_dict["vol_enabled"].value,
            "target_volume": widgets_dict["target_volume"].value,
            "lambda_volume": widgets_dict["lambda_volume"].value
        })
    
    # Add/update surface constraints
    surf_exists = False
    for entry in constraints["Surface"]:
        if entry["CellType"] == cell_type:
            entry.update({
                "enabled": widgets_dict["surf_enabled"].value,
                "target_surface": widgets_dict["target_surface"].value,
                "lambda_surface": widgets_dict["lambda_surface"].value
            })
            surf_exists = True
            break
    
    if not surf_exists and widgets_dict["surf_enabled"].value:
        constraints["Surface"].append({
            "CellType": cell_type,
            "enabled": widgets_dict["surf_enabled"].value,
            "target_surface": widgets_dict["target_surface"].value,
            "lambda_surface": widgets_dict["lambda_surface"].value
        })
    
    save_to_json(current_config(widgets_dict, celltype_entries, constraints))
    update_constraints_display()

def reset_tab(tab_name):
    if tab_name == "Metadata":
        widgets_dict["num_proc"].value = DEFAULTS["Metadata"]["num_processors"]
        widgets_dict["debug_freq"].value = DEFAULTS["Metadata"]["debug_output_frequency"]
    
    elif tab_name == "PottsCore":
        widgets_dict["x_slider"].value = DEFAULTS["PottsCore"]["dim_x"]
        widgets_dict["y_slider"].value = DEFAULTS["PottsCore"]["dim_y"]
        widgets_dict["z_slider"].value = DEFAULTS["PottsCore"]["dim_z"]
        widgets_dict["steps_input"].value = DEFAULTS["PottsCore"]["steps"]
        widgets_dict["anneal_input"].value = DEFAULTS["PottsCore"]["anneal"]
        widgets_dict["fluctuation_slider"].value = DEFAULTS["PottsCore"]["fluctuation_amplitude"]
        widgets_dict["flunct_fn_dropdown"].value = DEFAULTS["PottsCore"]["fluctuation_amplitude_function"]
        widgets_dict["boundary_x"].value = DEFAULTS["PottsCore"]["boundary_x"]
        widgets_dict["boundary_y"].value = DEFAULTS["PottsCore"]["boundary_y"]
        widgets_dict["boundary_z"].value = DEFAULTS["PottsCore"]["boundary_z"]
        widgets_dict["neighbor_order_input"].value = DEFAULTS["PottsCore"]["neighbor_order"]
        widgets_dict["seed_input"].value = ''
        widgets_dict["lattice_dropdown"].value = 'Square' if DEFAULTS["PottsCore"]["lattice_type"] == 'Cartesian' else 'Hexagonal'
        widgets_dict["offset_input"].value = DEFAULTS["PottsCore"]["offset"]
    
    elif tab_name == "CellType":
        global celltype_entries
        celltype_entries = DEFAULTS["CellType"].copy()
        update_celltype_display(celltype_entries, widgets_dict["celltype_display_box"])
        widgets_dict["constraints_celltype_dropdown"].options = [entry["Cell type"] for entry in celltype_entries]
    
    elif tab_name == "Constraints":
        global constraints
        constraints = DEFAULTS["Constraints"].copy()
        update_constraints_display()
    
    save_to_json(current_config(widgets_dict, celltype_entries, constraints))

def reset_to_defaults(_):
    reset_tab("Metadata")
    reset_tab("PottsCore")
    reset_tab("CellType")
    reset_tab("Constraints")

# Set up event handlers
widgets_dict["preset_dropdown"].observe(toggle_custom_input, names='value')
widgets_dict["add_button"].on_click(on_add_clicked)
widgets_dict["add_constraints_button"].on_click(on_add_constraints_clicked)

widgets_dict["reset_metadata_button"].on_click(lambda _: reset_tab("Metadata"))
widgets_dict["reset_potts_button"].on_click(lambda _: reset_tab("PottsCore"))
widgets_dict["reset_ct_button"].on_click(lambda _: reset_tab("CellType"))
widgets_dict["reset_constraints_button"].on_click(lambda _: reset_tab("Constraints"))
widgets_dict["reset_button"].on_click(reset_to_defaults)

# Set save triggers
for w in [widgets_dict[k] for k in [
    "num_proc", "debug_freq", "x_slider", "y_slider", "z_slider",
    "steps_input", "anneal_input", "fluctuation_slider", "flunct_fn_dropdown",
    "neighbor_order_input", "boundary_x", "boundary_y", "boundary_z",
    "lattice_dropdown", "offset_input", "seed_input"
]]:
    w.observe(lambda _: save_to_json(current_config(widgets_dict, celltype_entries, constraints)), names='value')

# Initialize constraints display
update_constraints_display()

In [8]:
# Cell 7: Main UI Assembly
# Create tabs
tabs = setup_tabs()

# Display everything
display(VBox([
    tabs,
    widgets_dict["reset_button"]
]))